### Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

### 1. Импортируем нужные библиотеки и задаём переменные

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [2]:
URL = 'https://habr.com/ru/all/'

In [3]:
KEYWORDS = ['python', 'парсинг']

In [4]:
res = requests.get(URL)
res

<Response [200]>

### 2. Собираем посты с заданного pages числа страниц

In [5]:
def get_all_posts(url, pages):
    '''В цикле пройдется по заданному количеству страниц и соберёт с каждой посты в список all_posts'''
    
    all_posts = []
    
    for i in range(1, pages):
        res = requests.get(f'{URL}page{i}/')
        time.sleep(0.5)
        
        soup = BeautifulSoup(res.text, 'html.parser')
        page_posts = soup.find_all('article', class_='post')
        all_posts += page_posts
        

    return all_posts

In [50]:
pages = 2
all_posts = get_all_posts(URL, pages)

### 3. Перебираем все посты на предмет совпадения ключевых слов

In [36]:
def get_time_from_post(post):
    '''Вернет для каждого поста корректные дату и время'''
    
    date = post.find('span', class_='post__time').text
    
    if 'сегодня' in date:
        today = datetime.today()
        date = date.replace('сегодня', today.strftime('%d %B %Y'))
    
    elif 'вчера' in date:
        yesterday = datetime.today() - timedelta(days=1)
        date = date.replace('вчера', yesterday.strftime('%d %B %Y'))
    
    date = datetime.strptime(date, '%d %B %Y в %H:%M')
        
    return date

In [37]:
def get_hubr_news_by_keywords(posts):
    '''Найдет в каждом посте текст preview, и пробежится по нему в поисках ключевых слов.
       При совпадении добавит строку в df.
    '''
    
    news = pd.DataFrame()
    for post in posts:
        preview_text = post.find('div', class_='post__text').text.lower()
        
        for word in KEYWORDS:
            if word in preview_text:
                date = get_time_from_post(post)
                title = post.find('a', class_='post__title_link').text
                link = post.find('a', class_='post__title_link').get('href')
                
                row = {'date' : date, 'title' : title, 'link' : link}
                news = pd.concat([news, pd.DataFrame([row])])
                break
                
    news = news.reset_index().drop('index', 1)
    return news

### 4. Собираем в df

In [38]:
keywords_news = get_hubr_news_by_keywords(all_posts)
keywords_news

""


#### Дополнительная часть (необязательная)¶
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [39]:
def get_hubr_news_by_keywords0(posts):
    '''Найдет в каждом посте текст preview, и пробежится по нему в поисках ключевых слов.
       Если не найдёт - будет смотреть в тексте страницы.
       При совпадении добавит строку в df.   
    '''
    
    news = pd.DataFrame()
    
    for post in posts:
        text_preview = post.find('div', class_='post__text').text.lower()
        
        for word in KEYWORDS: 
            if word in text_preview:
                link = post.find('a', class_='post__title_link').get('href')
                
                soup = BeautifulSoup(requests.get(link).text, 'html.parser')
                time.sleep(0.3)
                
                date = soup.find('span', class_='post__time').get('data-time_published')
                title = soup.find('span', class_='post__title-text').text
                text = soup.find('div', class_='post__text').text
                            
                date = datetime.strptime(date, '%Y-%m-%dT%H:%MZ')
                    
                row = {'date' : date, 'title' : title, 'link' : link, 'text' : text}
                news = pd.concat([news, pd.DataFrame([row])]) 
                break
                    
        # если ключевые слова не нашел в preview, то ищет внутри поста:   
        else:
            link = post.find('a', class_='post__title_link').get('href')
            
            soup = BeautifulSoup(requests.get(link).text, 'html.parser')
            time.sleep(0.3)
            
            text = soup.find('div', class_='post__text').text.lower()
            
            for word in KEYWORDS:
                if word in text:
                    date = soup.find('span', class_='post__time').get('data-time_published')
                    title = soup.find('span', class_='post__title-text').text
                    
                    date = datetime.strptime(date, '%Y-%m-%dT%H:%MZ')
                    
                    row = {'date' : date, 'title' : title, 'link' : link, 'text' : text}
                    news = pd.concat([news, pd.DataFrame([row])])
                    break
                    
                
    news = news.reset_index().drop('index', 1)
    return news

In [40]:
keywords_news0 = get_hubr_news_by_keywords0(all_posts)
keywords_news0

,date,title,link,text
0,2021-04-05 08:25:00,"Сборка не JVM-проектов, инструменты",https://habr.com/ru/post/550658/,в некоторых проектах сборке отводится роль зол...
1,2021-04-05 07:58:00,Нейродайджест: главное из области машинного об...,https://habr.com/ru/post/550726/,в марте было особенно много новостей про приме...
2,2021-04-05 07:45:00,No comments,https://habr.com/ru/company/macloud/blog/550608/,\n\r\n«комментарии должны составлять 5% от общ...
3,2021-04-05 06:25:00,Ломаем зашифрованный диск для собеседования от...,https://habr.com/ru/post/549578/,по мотивамчасть 0x00а что дальше?уважаемые хаб...


### Задание 2.¶
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [41]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

In [42]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

In [43]:
payload = {'emailAddresses': EMAIL}
headers = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast'
}

In [44]:
avast_responce = requests.post(url, json=payload, headers=headers)
avast_responce

<Response [200]>

In [51]:
avast_info = avast_responce.json()

In [46]:
breahes_dict = {}
for mail, breaches in avast_info['summary'].items():
    breahes_dict[mail] = breaches['breaches']
print(breahes_dict) 

{'xxx@x.ru': [3176, 12, 3, 3164, 2961, 15], 'yyy@y.com': [17110, 17670, 13094, 37177, 16768, 41, 2, 3587, 17009, 3, 13662, 3520, 15, 3669]}


In [47]:
avast_info_df = pd.DataFrame()

for mail, breaches_list in breahes_dict.items():
    for breach in breaches_list:
        
        date = datetime.strptime(avast_info['breaches'][f'{breach}']['publishDate'], '%Y-%m-%dT%H:%M:%SZ')
        site = avast_info['breaches'][f'{breach}']['site']
        description = avast_info['breaches'][f'{breach}']['description']
        
        row = {'почта' : mail, 'дата утечки' : date, 'источник утечки' : site, 'описание утечки' : description}
        avast_info_df = pd.concat([avast_info_df, pd.DataFrame([row])])

avast_info_df = avast_info_df.reset_index().drop('index', 1)

In [48]:
avast_info_df

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,2017-02-14,parapa.mail.ru,"In July and August 2016, two criminals execute..."
1,xxx@x.ru,2016-10-29,vk.com,Popular Russian social networking platform VKo...
2,xxx@x.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
3,xxx@x.ru,2017-02-14,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
4,xxx@x.ru,2017-01-31,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
5,xxx@x.ru,2016-10-23,imesh.com,"In June 2016, a cache of over 51 million user ..."
6,yyy@y.com,2020-01-03,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
7,yyy@y.com,2020-05-28,wishbone.io,"In January 2020, the online poll website Wishb..."
8,yyy@y.com,2017-11-04,myheritage.com,"In October 2017, a customer database belonging..."
9,yyy@y.com,2021-02-11,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m..."


#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [117]:
with open('token.txt', 'r') as file_object:
    TOKEN = file_object.read().strip()

In [118]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get'
VERSION = '5.130'

In [119]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': 'netology',
    'count': 50
}

In [120]:
res = requests.get(NEWSFEED_REQUEST, params)
res

<Response [200]>

In [121]:
posts = res.json()['response']['items']

In [122]:
posts_df = pd.DataFrame()
for post in posts:
    date = datetime.fromtimestamp(post['date'])
    text = post['text']
    
    row = {'дата поста': date, 'текст поста': text}
    posts_df = pd.concat([posts_df, pd.DataFrame([row])])
    
posts_df = posts_df.reset_index().drop('index', 1)
posts_df

,дата поста,текст поста
0,2021-04-02 10:18:00,"Пишем коды для веб-страниц, осваиваем языки пр..."
1,2021-04-05 10:16:00,🚀 Запустили бесплатный курс «Основы интернет-м...
2,2021-04-04 13:52:00,"Мы знаем, что найти работу не всегда легко 🤗 \..."
3,2021-04-03 15:25:00,*партнёрский пост* \n \n15 апреля состоится он...
4,2021-04-03 13:46:00,"Чтобы писать красиво, просто и понятно — недос..."
5,2021-04-01 18:46:00,Прокрастинация — это когда нужно работать здес...
6,2021-04-01 11:15:00,"Чтобы сменить специальность, нужно признать, ч..."
7,2021-03-31 15:32:32,Разыгрываем 10 доступов к библиотеке видеокурс...
8,2021-03-31 10:49:00,"Кажется, в SMM всё просто: написал пост, ответ..."
9,2021-03-30 17:49:00,В 2020 году сербская художница-перформанистка ...
